In [123]:
!pip install Pandas


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [126]:
from bs4 import BeautifulSoup as BS
import requests
import locale
import pandas as pd

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [4]:
html_text = requests.get('https://www.scrapethissite.com/pages/simple/').text

In [5]:
html_parsed = BS(html_text, 'html.parser')

## Extractiong Data

In [205]:
# Extractiong Data

country_list = html_parsed.find_all(class_='country-name')
population_list = html_parsed.find_all(class_='country-population')
area_list = html_parsed.find_all(class_='country-area')

## Processing Data

In [310]:
def calc_density(population, area):
    result = 0    
    if population == 0.00:
        result = population
    else:
        result = (population / area)

    return result


In [339]:
# Processing Data
countries = []
for country, population, area in zip(country_list, population_list, area_list):

    text_country = country.text.strip()
    float_population = float(population.text.strip())
    float_area = float(area.text.strip())
    
    dic = {'Country': text_country, 
           'Population': float_population, 
           'Area': float_area,
           'Density (hab/km²)': calc_density(float_population, float_area)
          }
    countries.append(dic)

In [340]:
countries
   

[{'Country': 'Andorra',
  'Population': 84000.0,
  'Area': 468.0,
  'Density (hab/km²)': 179.48717948717947},
 {'Country': 'United Arab Emirates',
  'Population': 4975593.0,
  'Area': 82880.0,
  'Density (hab/km²)': 60.033699324324324},
 {'Country': 'Afghanistan',
  'Population': 29121286.0,
  'Area': 647500.0,
  'Density (hab/km²)': 44.974959073359074},
 {'Country': 'Antigua and Barbuda',
  'Population': 86754.0,
  'Area': 443.0,
  'Density (hab/km²)': 195.83295711060947},
 {'Country': 'Anguilla',
  'Population': 13254.0,
  'Area': 102.0,
  'Density (hab/km²)': 129.94117647058823},
 {'Country': 'Albania',
  'Population': 2986952.0,
  'Area': 28748.0,
  'Density (hab/km²)': 103.90121051899263},
 {'Country': 'Armenia',
  'Population': 2968000.0,
  'Area': 29800.0,
  'Density (hab/km²)': 99.59731543624162},
 {'Country': 'Angola',
  'Population': 13068161.0,
  'Area': 1246700.0,
  'Density (hab/km²)': 10.482201812785755},
 {'Country': 'Antarctica',
  'Population': 0.0,
  'Area': 14000000.

# Without Pandas

## Top 10 countries with the highest density

In [341]:
heighest_10_dense_countries = sorted(countries, key=lambda x: x['Density (hab/km²)'], reverse=True)[:10]

print('-----------------------------------------')
print('Top 10 countries with the highest density')
print('-----------------------------------------\n')
for index, country in enumerate(heighest_10_dense_countries):
     density = country['Density (hab/km²)']
     print(f"{index+1}° - {country['Country']}: {locale.format_string('%0.2f', density, grouping=True)} hab/km²")


-----------------------------------------
Top 10 countries with the highest density
-----------------------------------------

1° - Monaco: 16,905.13 hab/km²
2° - Singapore: 6,786.59 hab/km²
3° - Hong Kong: 6,317.48 hab/km²
4° - Gibraltar: 4,289.85 hab/km²
5° - Vatican City: 2,093.18 hab/km²
6° - Sint Maarten: 1,782.33 hab/km²
7° - Macao: 1,768.50 hab/km²
8° - Maldives: 1,318.83 hab/km²
9° - Malta: 1,275.32 hab/km²
10° - Bermuda: 1,233.30 hab/km²


## Top 10 countries with the lowest density 

In [409]:
lowest_10_dense_countries = sorted(
    [country for country in countries if float(country['Density (hab/km²)']) > 0],
    key=lambda x: x['Density (hab/km²)']
)[:10]

print('-----------------------------------------')
print('Top 10 countries with the lowest density')
print('-----------------------------------------')
for index, country in enumerate(lowest_10_dense_countries):
    density = country['Density (hab/km²)']
    print(f"{index+1}° - {country['Country']}: {locale.format_string('%0.2f', density, grouping=True)} hab/km²")

-----------------------------------------
Top 10 countries with the lowest density
-----------------------------------------
1° - South Georgia and the South Sandwich Islands: 0.01 hab/km²
2° - French Southern Territories: 0.02 hab/km²
3° - Greenland: 0.03 hab/km²
4° - Svalbard and Jan Mayen: 0.04 hab/km²
5° - Falkland Islands: 0.22 hab/km²
6° - Pitcairn Islands: 0.98 hab/km²
7° - Western Sahara: 1.03 hab/km²
8° - Mongolia: 1.97 hab/km²
9° - French Guiana: 2.15 hab/km²
10° - Namibia: 2.58 hab/km²


# Pandas

In [410]:
# Create data frame
df = pd.DataFrame(countries)

## Converting a data frame into a table

In [418]:
df = df[['Country', 'Density (hab/km²)']]

## Top 10 countries with the highest density

In [412]:
def format_number(number):
    return '{:,.2f}'.format(number)

In [413]:
top_10_highest_density = df.nlargest(10, 'Density (hab/km²)')
top_10_highest_density['Density (hab/km²)'] = top_10_highest_density['Density (hab/km²)'].apply(format_number)

In [414]:
top_10_highest_density.head(10).style.set_table_styles([ 
    dict( selector='th', props=[('text-align', 'left')]),                                                
    dict( selector='td', props=[('text-align', 'left')])
])

,Country,Density (hab/km²)
137,Monaco,"16,905.13"
197,Singapore,"6,786.59"
94,Hong Kong,"6,317.48"
82,Gibraltar,"4,289.85"
235,Vatican City,"2,093.18"
210,Sint Maarten,"1,782.33"
147,Macao,"1,768.50"
154,Maldives,"1,318.83"
152,Malta,"1,275.32"
26,Bermuda,"1,233.30"


## Top 10 countries with the lowest density

In [431]:
filtered_df = df[(df['Density (hab/km²)'] > 0.00)]

In [432]:
top_10_lowest_density = filtered_df.nsmallest(10,'Density (hab/km²)')
top_10_lowest_density['Density (hab/km²)'] = top_10_lowest_density['Density (hab/km²)'].apply(format_number)

In [434]:
top_10_lowest_density.head(10).style.set_table_styles([ 
    dict( selector='th', props=[('text-align', 'left')]),                                                
    dict( selector='td', props=[('text-align', 'left')])
])

,Country,Density (hab/km²)
89,South Georgia and the South Sandwich Islands,0.01
215,French Southern Territories,0.02
83,Greenland,0.03
200,Svalbard and Jan Mayen,0.04
71,Falkland Islands,0.22
180,Pitcairn Islands,0.98
65,Western Sahara,1.03
146,Mongolia,1.97
79,French Guiana,2.15
159,Namibia,2.58
